In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    Forward
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
N_PATHS = 100_000
N_STOCKS = 1
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL = 0.2

In [4]:
N_DAYS

1095.0

In [5]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    dividends=np.array([0.0] * N_STOCKS),
    means=np.array([RF_RATE] * N_STOCKS),
    var_covar=np.array([VOL**2] * N_STOCKS),
)
underlyings.data

In [6]:
curve = ConstantRateCurve(currency="RUB", rate=RF_RATE)

In [8]:
forward = Forward(
    yield_curve=curve,
    start_date=start,
    end_date=end
)

In [9]:
pricer = GBMPricer(payoff_function=forward.payoff, random_seed=RANDOM_SEED)

In [10]:
forward.strike, np.exp(TILL_MATURITY * curve.get_rate(3.))

(1.0941742837052104, 1.0941742837052104)

In [11]:
pricer.price(
    spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
)

0.00010373860153460368

In [12]:
paths = pricer.get_paths(
    spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
).squeeze(2)
paths.shape

(100000, 757)

In [13]:
paths[:, -1].mean()

1.0942877918152374

In [14]:
(paths[:, -1] - forward.strike).mean()

0.00011350811002670527

In [15]:
forward.price()

array([0.])